In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import date, timedelta
from sqlalchemy import create_engine
engine = create_engine(sensitive-info-here)

In [2]:
# Obtaining first and last day of previous month

last_day_of_prev_month = date.today().replace(day=1) - timedelta(days=1)
start_day_of_prev_month = date.today().replace(day=1) - timedelta(days=last_day_of_prev_month.day)
from_date = start_day_of_prev_month.strftime("%Y-%m-%d")
to_date = last_day_of_prev_month.strftime("%Y-%m-%d")

# Obtaining previous month name
prev_month_name = last_day_of_prev_month.strftime('%B')

# Obtaining year
year = last_day_of_prev_month.strftime("%Y")

# Obtaining infoZ from endpoint

In [3]:
infoZ = pd.read_sql_query('select registration_number,serial, model from "myendpoint"',con=engine)

In [4]:
infoZ.columns = ['objectnumber','serial','program']

In [5]:
infoZ.set_index('objectnumber', inplace=True)

# Consolidate data extracted from my website

In [6]:
file_name = 'mywebsite_{}.xlsx'.format(prev_month_name)
df = pd.read_excel(file_name)

In [7]:
# Creates a copy of original dataframe
df_copy = df

In [8]:
# Information I have to remove from the xlsx file

df_copy = df_copy[df_copy.Duration != 'Redirected']
df_copy = df_copy[df_copy.Duration != 'On Air']

#The following information was removed in the past but now I have to include it. This is what is causing the problem. It is the only info from duration tha does not contain numbers
#df_copy = df_copy[df_copy.Duration != 'No History Data (searched last 3 months)']

df_copy = df_copy[df_copy.Duration != 'Programmed']
df_copy = df_copy[df_copy.Duration != 'unknown']
df_copy.Duration.replace('', np.nan, inplace=True)
df_copy.dropna(subset=['Duration'], inplace=True)


In [9]:
def format_date(text):
    months = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}
    if(len(text) == 17):
        date = text[13:17]+'-'+months[text[6:9]]+'-'+text[0:2]
    elif(len(text) == 16):
        date = text[12:16]+'-'+months[text[5:8]]+'-0'+text[0:1]
    return date

In [10]:
def format_timedelta(td):
    minutes, seconds = divmod(td.seconds + td.days * 86400, 60)
    hours, minutes = divmod(minutes, 60)
    return '{:d}:{:02d}'.format(hours, minutes)

In [11]:
#Formatting the Date
df_copy['Date'] = df_copy['Date'].apply(format_date)

In [12]:
df_copy['Date'] = pd.to_datetime(df_copy['Date'], format='%Y-%m-%d')

In [13]:
df_copy['Duration'] = pd.to_datetime(df_copy['Duration'], format='%H:%M').dt.time

In [14]:
df_copy['Duration'] = pd.to_timedelta(df_copy['Duration'].astype(str))

In [15]:
df_copy.index = df_copy['Date']
current_month = df_copy[from_date:to_date]

In [16]:
fh = current_month['Duração'].groupby(current_month['objectnumber']).sum()

In [17]:
fc = current_month.groupby(['objectnumber']).size()

In [18]:
fc = pd.DataFrame([fc])
fc = fc.T
fc.columns = ['FC']
fh = pd.DataFrame([fh])
fh = fh.T
fh.columns = ['FH']

In [19]:
fh['FH'] = fh['FH'].apply(format_timedelta)

In [20]:
fhfc = fh.join(fc)

In [21]:
fhfc_infoZ = fhfc.join(infoZ)

In [22]:
infoZ_fhfc = infoZ.join(fhfc)

In [23]:
result_file_name = 'mywebsite_{}_{}.xlsx'.format(prev_month_name,year)
fhfc_infoZ.to_excel(result_file_name)